In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path 
import missingno as msno
from tabulate import tabulate
from statistics import median, mean, quantiles
import pprint
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from datetime import datetime

In [74]:
path = Path(os.getcwd()) 
p_parent = path.parent
p_customer = str(p_parent) + '\data\olist_customers_dataset.csv'
p_geo = str(p_parent) + '\data\olist_geolocation_dataset.csv'
p_order_item = str(p_parent) + '\data\olist_order_items_dataset.csv'
p_order_payment = str(p_parent) + '\data\olist_order_payments_dataset.csv'
p_order_review = str(p_parent) + '\data\olist_order_reviews_dataset.csv'
p_orders = str(p_parent) + '\data\olist_orders_dataset.csv'
p_products = str(p_parent) + '\data\olist_products_dataset.csv'
p_sellers = str(p_parent) + '\data\olist_sellers_dataset.csv'
p_category = str(p_parent) + '\data\product_category_name_translation.csv'

customer = pd.read_csv(p_customer)
geo = pd.read_csv(p_geo)
order_item = pd.read_csv(p_order_item)
order_payment = pd.read_csv(p_order_payment)
order_review = pd.read_csv(p_order_review)
orders = pd.read_csv(p_orders)
products = pd.read_csv(p_products)
sellers = pd.read_csv(p_sellers)
category = pd.read_csv(p_category)

In [34]:
# Création d'un DF pour afficher les stats des données
def tstats (data) :
    output = []
    for col in data.columns:
    
        nonNull  = len(data) - np.sum(pd.isna(data[col]))
        nonNullprop = (nonNull / len(data[col]))*100
        unique = data[col].nunique()
        colType = str(data[col].dtype)
        output.append([col, colType, nonNull, round(nonNullprop, 1) , unique])
            
    
    df_stats = pd.DataFrame(output)
    df_stats.columns = ['nom colonne','dtype', 'valeur non null',"% de non null", 'nb_unique']

    print(tabulate(df_stats, headers='keys', tablefmt='psql'))

In [20]:
order_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [33]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [44]:
order_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [45]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [46]:
order_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [47]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [48]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [49]:
order_customer = orders.merge(customer, on="customer_id", how ='right')
order_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   customer_id                    99441 non-null  object
 1   customer_unique_id             99441 non-null  object
 2   customer_zip_code_prefix       99441 non-null  int64 
 3   customer_city                  99441 non-null  object
 4   customer_state                 99441 non-null  object
 5   order_id                       99441 non-null  object
 6   order_status                   99441 non-null  object
 7   order_purchase_timestamp       99441 non-null  object
 8   order_approved_at              99281 non-null  object
 9   order_delivered_carrier_date   97658 non-null  object
 10  order_delivered_customer_date  96476 non-null  object
 11  order_estimated_delivery_date  99441 non-null  object
dtypes: int64(1), object(11)
memory usage: 9.9+ MB


In [52]:
orderItems_products = order_item.merge(products, on ='product_id', how ='left').merge(sellers, on='seller_id', how = "left")
orderItems_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_id                    112650 non-null  object 
 1   order_item_id               112650 non-null  int64  
 2   product_id                  112650 non-null  object 
 3   seller_id                   112650 non-null  object 
 4   shipping_limit_date         112650 non-null  object 
 5   price                       112650 non-null  float64
 6   freight_value               112650 non-null  float64
 7   product_category_name       111047 non-null  object 
 8   product_name_lenght         111047 non-null  float64
 9   product_description_lenght  111047 non-null  float64
 10  product_photos_qty          111047 non-null  float64
 11  product_weight_g            112632 non-null  float64
 12  product_length_cm           112632 non-null  float64
 13  product_height

In [4]:
order_review = orders.merge(order_review, on ='order_id', how ='left')
order_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99992 entries, 0 to 99991
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       99992 non-null  object 
 1   customer_id                    99992 non-null  object 
 2   order_status                   99992 non-null  object 
 3   order_purchase_timestamp       99992 non-null  object 
 4   order_approved_at              99831 non-null  object 
 5   order_delivered_carrier_date   98199 non-null  object 
 6   order_delivered_customer_date  97005 non-null  object 
 7   order_estimated_delivery_date  99992 non-null  object 
 8   review_id                      99224 non-null  object 
 9   review_score                   99224 non-null  float64
 10  review_comment_title           11568 non-null  object 
 11  review_comment_message         40977 non-null  object 
 12  review_creation_date           99224 non-null 

In [18]:
#version courte des merges de dbs
df = orders.merge(customer, on="customer_id", how ='left')\
.merge(order_item, on= "order_id", how = 'left')\
.merge(order_payment, on ='order_id', how ='left')\
.merge(order_review, on ="order_id", how="left")\
.merge(sellers, on ='seller_id', how ="left")\
.merge(products, on = "product_id", how = 'left')\
.merge(category, on="product_category_name", how="left")

tstats(df)

+----+---------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                     | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+---------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                        | object  |            119143 |           100   |       99441 |
|  1 | customer_id_x                   | object  |            119143 |           100   |       99441 |
|  2 | order_status_x                  | object  |            119143 |           100   |           8 |
|  3 | order_purchase_timestamp_x      | object  |            119143 |           100   |       98875 |
|  4 | order_approved_at_x             | object  |            118966 |            99.9 |       90733 |
|  5 | order_delivered_carrier_date_x  | object  |            117057 |            98.2 |       81018 |
|  6 | order_delivered_customer_date_x | object  |            115722 |   

In [24]:
#version courte des merges de dbs
df = orders.merge(customer, on="customer_id", how ='outer')\
.merge(order_item, on= "order_id", how = 'outer')\
.merge(order_payment, on ='order_id', how ='outer')\
.merge(order_review, on ="order_id", how="outer")\
.merge(sellers, on ='seller_id', how ="outer")\
.merge(products, on = "product_id", how = 'outer')\
.merge(category, on="product_category_name", how="outer")

tstats(df)

+----+---------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                     | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+---------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                        | object  |            119143 |           100   |       99441 |
|  1 | customer_id_x                   | object  |            119143 |           100   |       99441 |
|  2 | order_status_x                  | object  |            119143 |           100   |           8 |
|  3 | order_purchase_timestamp_x      | object  |            119143 |           100   |       98875 |
|  4 | order_approved_at_x             | object  |            118966 |            99.9 |       90733 |
|  5 | order_delivered_carrier_date_x  | object  |            117057 |            98.2 |       81018 |
|  6 | order_delivered_customer_date_x | object  |            115722 |   

In [41]:
#version courte des merges de dbs
df = order_item.merge(orders, on="order_id", how ='inner')\
.merge(customer, on= "customer_id", how = 'inner')\
.merge(order_payment, on ='order_id', how ='inner')\
.merge(sellers, on ='seller_id', how ="inner")\
.merge(products, on = "product_id", how = 'inner')\
.merge(category, on="product_category_name", how="inner")\

df.drop('customer_id', axis = 1)
df.merge(order_review, on ="order_id", how="inner")
df= df.drop_duplicates()
tstats(df)

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |            115878 |           100   |       97255 |
|  1 | order_item_id                 | int64   |            115878 |           100   |          21 |
|  2 | product_id                    | object  |            115878 |           100   |       32328 |
|  3 | seller_id                     | object  |            115878 |           100   |        3033 |
|  4 | shipping_limit_date           | object  |            115878 |           100   |       92052 |
|  5 | price                         | float64 |            115878 |           100   |        5899 |
|  6 | freight_value                 | float64 |            115878 |           100   |     

In [35]:
tstats(order_review)

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |             99992 |           100   |       99441 |
|  1 | customer_id                   | object  |             99992 |           100   |       99441 |
|  2 | order_status                  | object  |             99992 |           100   |           8 |
|  3 | order_purchase_timestamp      | object  |             99992 |           100   |       98875 |
|  4 | order_approved_at             | object  |             99831 |            99.8 |       90733 |
|  5 | order_delivered_carrier_date  | object  |             98199 |            98.2 |       81018 |
|  6 | order_delivered_customer_date | object  |             97005 |            97   |     

In [43]:
tstats(orders)

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |             99441 |           100   |       99441 |
|  1 | customer_id                   | object  |             99441 |           100   |       99441 |
|  2 | order_status                  | object  |             99441 |           100   |           8 |
|  3 | order_purchase_timestamp      | object  |             99441 |           100   |       98875 |
|  4 | order_approved_at             | object  |             99281 |            99.8 |       90733 |
|  5 | order_delivered_carrier_date  | object  |             97658 |            98.2 |       81018 |
|  6 | order_delivered_customer_date | object  |             96476 |            97   |     

In [46]:
order_review_test = order_review.drop(["review_id"
                                      , "review_score"
                                      , "review_comment_title"
                                      , "review_comment_message"
                                      , "review_creation_date"
                                      , "review_answer_timestamp"], axis = 1) 

In [47]:
tstats(order_review_test)

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |             99992 |           100   |       99441 |
|  1 | customer_id                   | object  |             99992 |           100   |       99441 |
|  2 | order_status                  | object  |             99992 |           100   |           8 |
|  3 | order_purchase_timestamp      | object  |             99992 |           100   |       98875 |
|  4 | order_approved_at             | object  |             99831 |            99.8 |       90733 |
|  5 | order_delivered_carrier_date  | object  |             98199 |            98.2 |       81018 |
|  6 | order_delivered_customer_date | object  |             97005 |            97   |     

In [50]:
print(orders.equals(order_review_test)) 

False


In [52]:
pd.concat([orders,order_review_test]).drop_duplicates(keep=False)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [66]:
test_concat = pd.concat([orders,order_review_test]).drop_duplicates()
tstats(test_concat)

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |             99441 |           100   |       99441 |
|  1 | customer_id                   | object  |             99441 |           100   |       99441 |
|  2 | order_status                  | object  |             99441 |           100   |           8 |
|  3 | order_purchase_timestamp      | object  |             99441 |           100   |       98875 |
|  4 | order_approved_at             | object  |             99281 |            99.8 |       90733 |
|  5 | order_delivered_carrier_date  | object  |             97658 |            98.2 |       81018 |
|  6 | order_delivered_customer_date | object  |             96476 |            97   |     

In [54]:
orders.merge(order_review_test,indicator = True, how='left').loc[lambda x : x['_merge']!='both']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,_merge


In [61]:
m = orders.merge(order_review_test, on='order_id', how='outer', suffixes=['', '_'], indicator=True)
tstats(m)

+----+--------------------------------+----------+-------------------+-----------------+-------------+
|    | nom colonne                    | dtype    |   valeur non null |   % de non null |   nb_unique |
|----+--------------------------------+----------+-------------------+-----------------+-------------|
|  0 | order_id                       | object   |             99992 |           100   |       99441 |
|  1 | customer_id                    | object   |             99992 |           100   |       99441 |
|  2 | order_status                   | object   |             99992 |           100   |           8 |
|  3 | order_purchase_timestamp       | object   |             99992 |           100   |       98875 |
|  4 | order_approved_at              | object   |             99831 |            99.8 |       90733 |
|  5 | order_delivered_carrier_date   | object   |             98199 |            98.2 |       81018 |
|  6 | order_delivered_customer_date  | object   |             97005 |   

In [62]:
nbBoth = m["_merge"].count()
print(nbBoth)

99992


In [75]:
df = orders.merge(customer, on="customer_id", how ='left').merge(order_review, on = "order_id", how='left')
tstats(df)

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |             99992 |           100   |       99441 |
|  1 | customer_id                   | object  |             99992 |           100   |       99441 |
|  2 | order_status                  | object  |             99992 |           100   |           8 |
|  3 | order_purchase_timestamp      | object  |             99992 |           100   |       98875 |
|  4 | order_approved_at             | object  |             99831 |            99.8 |       90733 |
|  5 | order_delivered_carrier_date  | object  |             98199 |            98.2 |       81018 |
|  6 | order_delivered_customer_date | object  |             97005 |            97   |     

In [76]:
df = orders.merge(customer, on="customer_id", how ='left')\
.merge(order_item, on= "order_id", how = 'left')\
.merge(order_review, on ="order_id", how="left")\
.merge(order_payment, on ='order_id', how ='left')\
.merge(sellers, on ='seller_id', how ="left")\
.merge(products, on = "product_id", how = 'left')\
.merge(category, on="product_category_name", how="left")

tstats(df)

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |            119143 |           100   |       99441 |
|  1 | customer_id                   | object  |            119143 |           100   |       99441 |
|  2 | order_status                  | object  |            119143 |           100   |           8 |
|  3 | order_purchase_timestamp      | object  |            119143 |           100   |       98875 |
|  4 | order_approved_at             | object  |            118966 |            99.9 |       90733 |
|  5 | order_delivered_carrier_date  | object  |            117057 |            98.2 |       81018 |
|  6 | order_delivered_customer_date | object  |            115722 |            97.1 |     

In [78]:
df = orders.merge(customer, on="customer_id", how ='left')
tstats(df)

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |             99441 |           100   |       99441 |
|  1 | customer_id                   | object  |             99441 |           100   |       99441 |
|  2 | order_status                  | object  |             99441 |           100   |           8 |
|  3 | order_purchase_timestamp      | object  |             99441 |           100   |       98875 |
|  4 | order_approved_at             | object  |             99281 |            99.8 |       90733 |
|  5 | order_delivered_carrier_date  | object  |             97658 |            98.2 |       81018 |
|  6 | order_delivered_customer_date | object  |             96476 |            97   |     

In [79]:
customer_groupby = df.groupby(['customer_unique_id'])['order_id'].count()
print(customer_groupby.value_counts().sort_index())

1     93099
2      2745
3       203
4        30
5         8
6         6
7         3
9         1
17        1
Name: order_id, dtype: int64
